#  ChromaDB 사용해보기

## 1. 필요 라이브러리 및 데이터 준비
ChromaDB를 사용하기 위해 필요한 라이브러리를 설치하고, 실험에 사용할 데이터를 준비합니다.   
데이터는 WellnessChatbot에서 제공해주는 위로 대화 데이터를 사용합니다.

In [1]:
!pip install chromadb sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.7/427.7 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import chromadb
client = chromadb.PersistentClient()

In [3]:
answers = client.create_collection(
    name="answers"
)

In [4]:
!git clone https://github.com/eunjiinkim/WellnessChatbot

Cloning into 'WellnessChatbot'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 84 (delta 35), reused 22 (delta 3), pack-reused 0
Receiving objects: 100% (84/84), 5.67 MiB | 6.15 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [5]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("./WellnessChatbot/chatbot_dataset.csv")
df.sample(5)

,user,system,sentiment
160678,남이랑 같이 밥 먹으면 속이 불편해져요,다이어트를 하지 않는데도 몸무게가 급격하게 감소하셨나요?,식욕저하
92040,심하다 싶을 정도로 잠이 안 온다,하루종일 잠만 자게 되시나요?,불면
272302,항상 스트레스 받아 있는 상태다 보니까… 뭔가 맨날 예민하고 날 서 있고…,하루에 30분도 차분히 앉아있지 못하고 뭔가를 해야 한다는 조바심이 느껴지시나요?,초조함
14880,손발이 떨릴 정도로 심하진 않아요,"그렇군요. 불안을 줄이는 방법으로 꾸준한 운동이나 명상, 규칙적인 수면이 도움이 될...",부정답변
83332,계속 얕게밖에 못 자는 이유를 모르겠어,잠을 자면서 자주 뒤척거리나요?,불면


In [6]:
df

,user,system,sentiment
0,12시 땡!,하루가 또 가네요.,일상
1,1지망 학교 떨어졌어,위로해 드립니다.,일상
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,일상
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,일상
4,PPL 심하네,눈살이 찌푸려지죠.,일상
...,...,...,...
333912,일주일정도 됐는데 점점 심해지는 것 같아요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,증상악화
333913,약을 끊었더니 증상이 예전보다 더 심해진것 같아.,너무 심하시면 병원을 다시 가보는 건 어떨까요?,증상악화
333914,한국에 돌아오고 나서 증상이 악화된 거 같아.,상태가 더 안 좋아지셨군요. 걱정이 되네요.,증상악화
333915,약을 끊었더니 증상이 예전보다 더 심해진 것 같아.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,증상악화


## 2. 벡터 데이터로 변환

허깅페이스의 한국어 모델을 이용하여 준비합니다.   
예시: snunlp/KR-SBERT-V40K-klueNLI-augSTS

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/elicer/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


텍스트 데이터를 임베딩 벡터로 변환하여 ChromaDB로 만든 콜랙션에 넘겨줍니다.

In [8]:

ids = []
metadatas = []
embeddings = []

for row in tqdm(list(df.iterrows())[:2000]):
    index = row[0]
    query = row[1].user
    answer = row[1].system

    metadata = {
        "query": query,
        "system": answer
    }

    embedding = model.encode(query, normalize_embeddings=True)

    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)

chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size

    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]

    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)


100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


## 3. ChromaDB 사용해보기
ChromaDB로 인덱스를 만들고, 변환된 벡터 데이터를 ChromaDB에 저장한 후 검색해봅니다.

In [15]:
result = answers.query(
    query_embeddings=model.encode("어제 여자친구랑 헤어졌다", normalize_embeddings=True).tolist(),
    n_results=3
)


print(result)

{'ids': [['488', '265', '726']], 'distances': [[0.6964139938354492, 0.7923784852027893, 0.8181809186935425]], 'metadatas': [[{'query': '나 몰래 사귀는 거 같애', 'system': '눈치가 빠르시군요.'}, {'query': '괜찮은 사람인데 사귀긴 싫어', 'system': '남자사람친구, 여자사람친구 하세요.'}, {'query': '남자친구 오늘 좀 질린다.', 'system': '결단은 빠를수록 좋아요.'}]], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


## 4. 로컬 피씨에 저장해서 사용하기(로컬 피씨로 진행)
chromadb.PersistentClient 객체는 원하는 경로에 있는 데이터를 불러오거나, 원하는 경로에 DB 데이터를 만들어, client 객체를 만들어줍니다.

In [16]:
client = chromadb.PersistentClient(path="newDB")

위와 같이 사용하면, 만들어진 데이터가 저장되며, 나중에 필요할 시 편리하게 벡터 DB를 불러와 사용해볼 수 있습니다.

ChromaDB는 서버로서 동작하여, 여러 사람이 동시에 접속하여 유사 벡터를 검색할 수 있도록 서비스를 제공하고 있습니다.    
   
서버로 사용하여 접속하고 싶을 때,

서버에서 해당 명령 실행
```shell
chroma run --path /db_path
```
이후 피씨에서 아래 명령어 실행(ip_address는 실제 자신의 IP주소를 적습니다. 예시: localhost )

In [19]:
import chromadb
#chroma_client = chromadb.HttpClient(host='{ip_address}', port=8000)
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

ValueError: Could not connect to a Chroma server. Are you sure it is running?